In [1]:
# Create monthly report url

# To run this directly from the command line, 
# jupyter nbconvert --execute "USDA Monthly Report.ipynb" --to asciidoc

from datetime import datetime, timedelta, date
# from urllib.parse import urlencode
import urllib.parse

report_date = '11/30/2023'

end = datetime.strptime(report_date, '%m/%d/%Y')
start = date(end.year, end.month, 1)


# from breeze import breeze
# breeze_api = breeze.BreezeApi(
#     breeze_url = ncc_url,
#     api_key='YourApiKey')

ncc_url = r"https://newmarketchurch.breezechms.com"
base_url = ncc_url + r"/events/reports/243292472#/"


import urllib.parse
params = {
    'start_date': urllib.parse.quote(start.strftime('%m/%d/%Y'), safe='/'),
    'end_date': urllib.parse.quote(end.strftime('%m/%d/%Y'), safe='/'),
    'event_ids': '2949650',
    'attendance_status': 'attended',
    'attendance_operator': 'any',
    'page': 'spreadsheet',
}

print("Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.")
print(base_url + urllib.parse.urlencode(params))

Download the spreadsheet from the following link. Copy the full path into the 'attendancereport' below.
https://newmarketchurch.breezechms.com/events/reports/243292472#/start_date=11%2F01%2F2023&end_date=11%2F30%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet


In [2]:
# This is the file that was downloaded from https://newmarketchurch.breezechms.com/events/reports/243292472#/&start_date=10%2F01%2F2023&end_date=10%2F31%2F2023&event_ids=2949650&attendance_status=attended&attendance_operator=any&page=spreadsheet

attendancereport = r"C:\Users\ralph\Downloads\FoodPantry-attendance-11-01-2023.xlsx"
peopleexport = r"C:\Users\ralph\Downloads\newmarketchurch-people-11-01-2023 (2).xlsx"

# Paste this file into this month's tab in https://docs.google.com/spreadsheets/d/1hAdgMh7_m73L--bR-rUT2dZPMwEJ8s2e/edit#gid=1826328980
paste_file = r"C:\Users\ralph\Downloads\paste_me.csv"

import numpy as np
import pandas as pd

# Read in the downloaded Excel file 
attendance = pd.read_excel(attendancereport)
people = pd.read_excel(peopleexport)
# print('{count} people in input.'.format(count = len(attendance.index)))

In [3]:
attendance.columns

Index(['Breeze ID', 'First Name', 'Last Name', 'Oct 5, 2023', 'Oct 12, 2023',
       'Oct 19, 2023', 'Oct 26, 2023', 'Person Totals'],
      dtype='object')

In [4]:
attendance = attendance[[
    'Breeze ID', 
    'First Name', 
    'Last Name', 
    'Person Totals'
]]
display(attendance)

,Breeze ID,First Name,Last Name,Person Totals
0,47038995.0,Samantha,Adario,2
1,46986494.0,Tomi,Adewuyi,1
2,44557131.0,Patricia,Alielouahed,3
3,43643318.0,Roberta,Anderson,4
4,45830007.0,Donald,April,4
...,...,...,...,...
169,45883734.0,Christy,Wyatt,4
170,30397680.0,Orson P.,Yancey,4
171,46915460.0,Christopher,Young,1
172,43643495.0,Patricia,Zwearcan,3


In [5]:
# people.columns

In [6]:
people = people[[
    'Breeze ID',
    'Date of First Contact',
    'How many people live in your household? (including yourself)',
    'Total number of people aged 18 or under in household?',
    'Total number of people between the ages of 19-59 in household?',
    'Total number of people aged 60 or older in Household?',
    'City'
]]

# people['Total number of people aged 18 or under in household?'] = people['Total number of people aged 18 or under in household?'].astype(int)

In [7]:
report = pd.merge(attendance, people, on = 'Breeze ID', how = "inner")
report.fillna(0, inplace=True)
print('{count} people in report.'.format(count = len(report.index)))
# report[~report.map(np.isreal)['Total number of people aged 18 or under in household?']]
# report[~report.map(np.isreal)['Total number of people between the ages of 19-59 in household?']]
# report[~report.map(np.isreal)['Total number of people aged 60 or older in Household?']]

# display(report)

173 people in report.


In [8]:
report.to_csv(paste_file, index=False)

In [9]:
print("Missing 'Total in house' for")
display(report.loc[report['How many people live in your household? (including yourself)'] == 0])

Missing 'Total in house' for


,Breeze ID,First Name,Last Name,Person Totals,Date of First Contact,How many people live in your household? (including yourself),Total number of people aged 18 or under in household?,Total number of people between the ages of 19-59 in household?,Total number of people aged 60 or older in Household?,City


In [10]:
print("Age breakdowns do not add up to total for")
display(report.loc[report['How many people live in your household? (including yourself)'] != 
        report['Total number of people aged 18 or under in household?'] + 
        report['Total number of people between the ages of 19-59 in household?'] + 
        report['Total number of people aged 60 or older in Household?']
])

Age breakdowns do not add up to total for


,Breeze ID,First Name,Last Name,Person Totals,Date of First Contact,How many people live in your household? (including yourself),Total number of people aged 18 or under in household?,Total number of people between the ages of 19-59 in household?,Total number of people aged 60 or older in Household?,City
